# Installing PySpark

In [2]:
pip install pyspark                                                  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 47.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e0043a0ca84a310620c63ece8774b05f42057648f2a599b4f842436936c863d4
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


# Improting Necessary Libraries

In [3]:
#Initializing PySpark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F


#Spark Config
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('Test').getOrCreate()
spark.sparkContext.getConf().getAll();

# Reading the CSV File

In [ ]:
df1 = spark.read.format("csv").option("header","true").option("inferschema","True").load("/content/UIDAI-ENR-DETAIL-20170308.csv") # Reading the dataset fromm the source
df1.show()
df1.printSchema()                                                                                                                  # printing the schema columns and its data types

In [12]:
df = df1.select([F.col(col).alias(col.replace(' ', '_')) for col in df1.columns]) # Removing the spaces in between the column Names and replacing it with under score ' _ ' 
df.show()
df.printSchema()                                                                  # printing the schema columns and its data types

                                                                                  # extracting number of columns from the Dataframe by using spark sql unctions
row = df.count()                                                                  # total number of rows in dataframe by using spark sql unctions
column = len(df.columns)                                                          # total number of columns in dataframe by using spark sql unctions
print(f'Number of Records are: {row}')
print(f'Number of Attributes are: {column}')

+--------------------+--------------------+-------------+----------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|           Registrar|    Enrolment_Agency|        State|  District|Sub_District|Pin_Code|Gender|Age|Aadhaar_generated|Enrolment_Rejected|Residents_providing_email|Residents_providing_mobile_number|
+--------------------+--------------------+-------------+----------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|      Allahabad Bank|A-Onerealtors Pvt...|Uttar Pradesh| Allahabad|        Meja|  212303|     F|  7|                1|                 0|                        0|                                1|
|      Allahabad Bank|Asha Security Gua...|Uttar Pradesh| Sonbhadra| Robertsganj|  231213|     M|  8|                1|                 0|                        0|                                0|
|    

In [19]:
col = df.columns                                                                 # getting the list of column names
print('List of column names: ')                                                  # Printing columns names
rdd1 = sc.parallelize(col)                                                       # stoting the columns in the rdd
rdd1.collect()                                         

List of column names: 


['Registrar',
 'Enrolment_Agency',
 'State',
 'District',
 'Sub_District',
 'Pin_Code',
 'Gender',
 'Age',
 'Aadhaar_generated',
 'Enrolment_Rejected',
 'Residents_providing_email',
 'Residents_providing_mobile_number']

# Create a dataframe with Total Aadhaar's generated for each state

In [45]:
# Uisng filter function we can select the column and it records,grouping the records by states and counting only aadhar geneated values and arranging them in decscending orrder.
Aadhar_count = df.filter(col("Aadhaar_generated") == 1).groupBy("State").agg(F.count("Aadhaar_generated").alias("Toatal_aadhar")).sort(F.desc("Toatal_aadhar"))
print("Total Aadhaar's generated for each state ")
Aadhar_count.show(20,truncate=False)

Total Aadhaar's generated for each state 
+--------------+-------------+
|State         |Toatal_aadhar|
+--------------+-------------+
|Bihar         |57652        |
|Uttar Pradesh |52652        |
|West Bengal   |43832        |
|Madhya Pradesh|27491        |
|Rajasthan     |21304        |
|Gujarat       |18358        |
|Tamil Nadu    |16464        |
|Maharashtra   |15234        |
|Karnataka     |12167        |
|Kerala        |9955         |
|Odisha        |9105         |
|Delhi         |5816         |
|Jharkhand     |5798         |
|Punjab        |4944         |
|Uttarakhand   |4341         |
|Haryana       |3936         |
|Andhra Pradesh|3519         |
|Chhattisgarh  |3409         |
|Telangana     |2953         |
|Assam         |2742         |
+--------------+-------------+
only showing top 20 rows



# Create a dataframe with the Total Aadhaar's generated by each enrollment agency

In [39]:
# Uisng filter function we can select the column and it records,grouping the records by Enrolment_Agency  and counting only aadhar geneated values and arranging them in decscending orrder.
Enrollment_agencys = df.filter(col("Aadhaar_generated") == 1).groupBy("Enrolment_Agency").agg(F.count("Aadhaar_generated").alias("Total_aadhar")).sort(F.desc("Total_aadhar"))
Enrollment_agencys.show(10,truncate=False)

+----------------------------------------+------------+
|Enrolment_Agency                        |Total_aadhar|
+----------------------------------------+------------+
|CSC SPV                                 |72155       |
|SRM Education And Social Welfare Society|13431       |
|SREI INFRASTRUCTURE FINANCES L          |12929       |
|AKSH OPTIFIBRE LIMITED                  |9828        |
|Akshaya                                 |9624        |
|TAMILNADU ARASU CABLE TV CORPORATION LTD|9624        |
|Rajcomp Info Services Ltd               |8752        |
|MPOnline Limited                        |7554        |
|CMS Computers Ltd                       |6895        |
|IAP COMPANY Pvt. Ltd                    |5765        |
+----------------------------------------+------------+
only showing top 10 rows



# Create dataframe with the top 10 districts with maximum Aadhaar's generated for both Male and females?

In [43]:
# Males
# # Uisng filter function we can select the column and it records,grouping the records by District and counting only aadhar geneated values and arranging them in decscending orrder

Males = df.filter("Gender == 'M'").groupBy("District").agg(F.count("Aadhaar_generated").alias("Total_aadhar")).sort(F.desc("Total_aadhar"))
print('Top 10 districts with maximum Aadhaar generated for Male : ')
Males.show(10,truncate=False)

# Females
# Uisng filter function we can select the column and it records,grouping the records by District and counting only aadhar geneated values and arranging them in decscending orrder.

Females=df.filter(("Gender == 'F'")).groupBy("District").agg(F.count("Aadhaar_generated").alias("Total_aadhar")).sort(F.desc("Total_aadhar"))
print('Top 10 districts with maximum Aadhaar generated for Female : ')
Females.show(10,truncate=False)


Top 10 districts with maximum Aadhaar generated for Male : 
+-----------------+------------+
|District         |Total_aadhar|
+-----------------+------------+
|Barddhaman       |4276        |
|North 24 Parganas|3772        |
|South 24 Parganas|3630        |
|Bhagalpur        |3543        |
|Patna            |3485        |
|Nadia            |3460        |
|Murshidabad      |3018        |
|Gaya             |2915        |
|Kolkata          |2678        |
|Katihar          |2622        |
+-----------------+------------+
only showing top 10 rows

Top 10 districts with maximum Aadhaar generated for Female : 
+-----------------+------------+
|District         |Total_aadhar|
+-----------------+------------+
|North 24 Parganas|3121        |
|Barddhaman       |2859        |
|South 24 Parganas|2448        |
|Patna            |1766        |
|Bhagalpur        |1744        |
|Nadia            |1673        |
|Jalpaiguri       |1509        |
|Gaya             |1487        |
|Murshidabad      |1399    

# Create a dataframe with Total Aadhaar generated for the top 10 least states

In [44]:
# Uisng filter function we can select the column and it records,grouping the records by states and counting only aadhar geneated values and arranging them in decscending orrder.
Aadhar_count = df.filter(col("Aadhaar_generated") == 1).groupBy("State").agg(F.count("Aadhaar_generated").alias("Toatal_aadhar")).sort(F.asc("Toatal_aadhar"))
print("Total Aadhaar's generated for each state ")
Aadhar_count.show(10,truncate=False)

Total Aadhaar's generated for each state 
+---------------------------+-------------+
|State                      |Toatal_aadhar|
+---------------------------+-------------+
|Lakshadweep                |4            |
|Andaman and Nicobar Islands|5            |
|Others                     |12           |
|Sikkim                     |43           |
|Puducherry                 |69           |
|Dadra and Nagar Haveli     |85           |
|Daman and Diu              |89           |
|Chandigarh                 |162          |
|Meghalaya                  |237          |
|Nagaland                   |289          |
+---------------------------+-------------+
only showing top 10 rows



# For which age, most Aadhar cards have declined?

In [52]:
# Uisng filter function we can select the column and it records,grouping the records by states and counting only aadhar geneated values and arranging them in decscending orrder.
Age = df.filter(col("Aadhaar_generated") == 0).groupBy("Age").agg(F.count("Aadhaar_generated").alias("Toatal_aadhar_Decline")).sort(F.desc("Toatal_aadhar_Decline"))
print('Most Aadharcard generations decline in this age Categoery:  ')
Age.show(10,truncate=False)

Most Aadharcard generations decline in this age Categoery:  
+---+---------------------+
|Age|Toatal_aadhar_Decline|
+---+---------------------+
|4  |1729                 |
|3  |1492                 |
|2  |1389                 |
|1  |1294                 |
|0  |1087                 |
|5  |863                  |
|6  |794                  |
|7  |724                  |
|8  |612                  |
|9  |529                  |
+---+---------------------+
only showing top 10 rows

